# Notebook con queries

Se importan las librerias necesarias.

In [83]:
import sqlite3
import pandas as pd

Se establece conexión con los archivos sql/ bases de datos, del proyecto.

In [84]:
# Conectar a la base de datos SQLite
conn = sqlite3.connect("data_detectives.db")
cursor = conn.cursor()
print("Conexión establecida.")

#Crear las tablas usando create.sql
with open("create.sql", "r", encoding="utf-8") as f:
    cursor.executescript(f.read())
conn.commit()
print("Tablas creadas.")

# Insertar los datos desde insert.sql
# Para archivos grandes, se hace por bloques (batch)
batch_size = 1000
batch = []

with open("insert.sql", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            batch.append(line)
        if len(batch) >= batch_size:
            cursor.executescript("\n".join(batch))
            conn.commit()
            batch = []

if batch:
    cursor.executescript("\n".join(batch))
    conn.commit()
print("Datos insertados correctamente.")

Conexión establecida.
Tablas creadas.
Datos insertados correctamente.


Se generan distintos dataframes para poder visualizar de mejor formas los datos y poder trabajar con ellos posteriomente.

In [85]:
# Conectar a la base de datos
# Necesario al ser un archivo .py y no .sql
conn = sqlite3.connect("data_detectives.db")

# Ejecutar las queries y guardar cada resultado en un DataFrame
df_regions = pd.read_sql_query("SELECT * FROM regions;", conn)
df_species = pd.read_sql_query("SELECT * FROM species;", conn)
df_climate = pd.read_sql_query("SELECT * FROM climate;", conn)
df_observations = pd.read_sql_query("SELECT * FROM observations;", conn)

# Queries  
Misión 1  
Obtener primeras 10 filas de df_observations.


In [86]:
# Codigo adaptado en cada querie
# Queremos primera 10 filas de df_observations
df_mision1 = pd.read_sql_query("""
SELECT * FROM observations
LIMIT 10;
""", conn)
# Imprimimos
df_mision1

,id,species_id,region_id,observer,observation_date,latitude,longitude,count
0,1,1,1,obsr297898,1992-07-07,-16.820152,145.63625,0
1,2,2,2,obsr1375321,1997-08-08,-33.126945,151.61362,0
2,3,3,1,obsr1898605,1981-12-13,-20.594380,139.54472,0
3,4,4,2,obsr166621,1999-10-17,-34.652440,150.80241,0
4,5,2,2,obsr797550,1999-11-13,-33.740300,151.08450,0
5,6,5,3,obsr340454,1997-05-10,-37.997130,144.56042,10
6,7,6,2,obsr1375321,1973-10-22,-37.439445,149.96638,15
7,8,7,3,obsr1437445,1995-01-07,-37.785960,145.01482,0
8,9,8,2,obsr631796,1966-12-10,-33.813310,150.90550,0
9,10,9,3,obsr2831785,1986-12-27,-37.227900,147.92300,0


MISIÓN 2  
Selección de los id de regiones.

In [87]:
# OJO:seguimos con df_observations
# Con AS cambiamos nombres
df_mision2 = pd.read_sql_query("""
SELECT DISTINCT region_id AS id_regiones
FROM observations;
""", conn)

df_mision2

,id_regiones
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


MISIÓN 3  
Conteo de las distintas especies.

In [88]:
# COUNT es comando de conteo en sql
# DISTINCT evita repetición de especie
df_mision3 = pd.read_sql_query("""
SELECT COUNT(DISTINCT species_id) AS conteo_especies
FROM observations;
""", conn)

df_mision3

,conteo_especies
0,270


Misión 4  

Conteo de observaciones de la región 2.

In [89]:
# Comando WHERE establece condición se usa antes de agrupar
df_mision4 = pd.read_sql_query("""
SELECT COUNT(*) AS conteo_region_2
FROM observations
WHERE region_id = 2;
""", conn)

df_mision4

,conteo_region_2
0,228


Misión 5  
Observaciones del 1998-08-08.

In [90]:
df_mision5 = pd.read_sql_query("""
SELECT *
FROM observations
WHERE observation_date = '1998-08-08';
""", conn)

df_mision5

,id,species_id,region_id,observer,observation_date,latitude,longitude,count
0,496,79,1,obsr202543,1998-08-08,-17.285713,145.62866,0


Misión 6  
Región con más observaciones.

In [91]:
# El conteo es modificado por los comandoS GROUP BY y ORDER
df_mision6 = pd.read_sql_query("""
SELECT region_id, COUNT(*) AS conteo_total
FROM observations
GROUP BY region_id
ORDER BY conteo_total DESC;
""", conn)

df_mision6

,region_id,conteo_total
0,2,228
1,1,81
2,3,50
3,7,45
4,9,20
5,14,11
6,8,10
7,19,6
8,16,6
9,13,6


Misión 7  
Las 5 especies más frecuentes.

In [92]:
df_mision7 = pd.read_sql_query("""
SELECT species_id, COUNT(*) AS especies_totales
FROM observations
GROUP BY species_id
ORDER BY especies_totales DESC
LIMIT 5;
""", conn)

df_mision7

,species_id,especies_totales
0,10,10
1,91,8
2,63,7
3,50,7
4,31,7


Misión 8  
Especies con menos de 5 registros.

In [93]:
# Comando HAVING se usa después de agrupar datos
df_mision8 = pd.read_sql_query("""
SELECT species_id AS id_especie, COUNT(*) AS especies_vistas
FROM observations
GROUP BY species_id
HAVING especies_vistas < 5
ORDER BY especies_vistas DESC;
""", conn)

df_mision8

,id_especie,especies_vistas
0,202,4
1,123,4
2,99,4
3,92,4
4,88,4
...,...,...
244,16,1
245,14,1
246,11,1
247,6,1


Misión 9  
Observadores con más observaciones.

In [94]:
df_mision9 = pd.read_sql_query("""
SELECT observer AS observador, COUNT(*) AS observaciones_hechas
FROM observations
GROUP BY observer
ORDER BY observaciones_hechas DESC
LIMIT 5;
""", conn)

df_mision9

,observador,observaciones_hechas
0,obsr453532,72
1,obsr1375321,20
2,obsr450704,15
3,obsr450476,13
4,obsr438330,13


Misión 10  
Región de cada observación.

In [95]:
df_mision10 = pd.read_sql_query("""
SELECT observations.id, regions.name AS nombre_region, observations.observation_date AS fecha_observacion
FROM observations
JOIN regions ON observations.region_id = regions.id;
""", conn)

df_mision10 

,id,nombre_region,fecha_observacion
0,1,Queensland,1992-07-07
1,2,New South Wales,1997-08-08
2,3,Queensland,1981-12-13
3,4,New South Wales,1999-10-17
4,5,New South Wales,1999-11-13
...,...,...,...
495,496,Queensland,1998-08-08
496,497,New South Wales,1979-11-23
497,498,Buenos Aires,1995-02-12
498,499,New South Wales,1983-03-05


Comentario comando JOIN en querie 10:
JOIN combina filas de dos tablas en base a una condición.

La condición está en ON observations.region_id = regions.id.

Esto significa: “une cada observación con la región cuya id coincida con region_id de la observación”.

Misión 11  
Nombre científico de cada especie, incluyendo la fecha de la observación.

In [96]:
df_mision11 = pd.read_sql_query("""
SELECT observations.id, species.scientific_name AS nombre_especie, observations.observation_date AS fecha_observacion
FROM observations
JOIN species ON observations.species_id = species.id;
""", conn)

df_mision11 

,id,nombre_especie,fecha_observacion
0,1,Cacatua galerita,1992-07-07
1,2,Sericornis frontalis,1997-08-08
2,3,Lalage tricolor,1981-12-13
3,4,Egretta novaehollandiae,1999-10-17
4,5,Sericornis frontalis,1999-11-13
...,...,...,...
495,496,Microcarbo melanoleucos,1998-08-08
496,497,Ninox boobook,1979-11-23
497,498,Anumbius annumbi,1995-02-12
498,499,Manorina melanocephala,1983-03-05


Misión 12  
Especie más observada por región.

In [97]:
df_mision12 = pd.read_sql_query("""
SELECT regions.name AS nombre_region, species.scientific_name AS nombre_especie, COUNT(*) AS conteo_total
FROM observations
JOIN species ON observations.species_id = species.id
JOIN regions ON observations.region_id = regions.id
GROUP BY nombre_region, nombre_especie
ORDER BY nombre_region, conteo_total DESC;
""", conn)

df_mision12 


,nombre_region,nombre_especie,conteo_total
0,Buenos Aires,Spatula platalea,2
1,Buenos Aires,Ardea cocoi,2
2,Buenos Aires,Zenaida auriculata,1
3,Buenos Aires,Serpophaga subcristata,1
4,Buenos Aires,Pluvialis dominica,1
...,...,...,...
358,Victoria,Aquila audax,1
359,Victoria,Antigone rubicunda,1
360,Victoria,Anas castanea,1
361,Victoria,Acridotheres tristis,1


Paso final

In [98]:
#Cerramos conexión
conn.close()
print("Conexión a la base de datos cerrada.")

Conexión a la base de datos cerrada.
